In [ ]:
# Defining head, discharge and power values for the vertices
psh.head_discharge_limits = pd.DataFrame(columns=psh.hydro_units, index=['max', 'min'])
psh.limhead_discharge_limitsits.loc['max', 'head'] = 10
psh.head_discharge_limits.loc['min', 'head'] = 4
psh.head_discharge_limits.loc['max', 'discharge'] = 30
psh.head_discharge_limits.loc['min', 'discharge'] = 15

efficieny = 0.98

head_interval = (psh.max_head - psh.min_head) / (len(psh.subregions) / 2)
discharge_interval = (psh.max_discharge - psh.min_discharge) / len(psh.subregions)

h_vertices = np.zeros((2,2,10,3))
q_vertices = np.zeros((2,2,10,3))

# head vertices
for i in cascaded_reservoirs:
    for j in hydro_units[i]:
        for l in subregions:
            if l % 2 == 0:
                h_vertices[i,j,l,0] = min_head + l / 2 * head_interval # bottom left
                h_vertices[i,j,l,1] = min_head + l / 2 * head_interval + head_interval # bottom right
                h_vertices[i,j,l,2] = h_vertices[i,j,l,0] # top left

                h_vertices[i,j,l+1,0] = h_vertices[i,j,l,0] # top left
                h_vertices[i,j,l+1,1] = h_vertices[i,j,l,1]# top right
                h_vertices[i,j,l+1,2] = h_vertices[i,j,l,1] # bottom right

# discharge vertices
for i in cascaded_reservoirs:
    for j in hydro_units[i]:
        for l in subregions:
            if l % 2 == 0:
                q_vertices[i,j,l,0] = min_discharge # bottom left
                q_vertices[i,j,l,1] = min_discharge # bottom right
                q_vertices[i,j,l,2] = max_discharge # top left

                q_vertices[i,j,l+1,0] = max_discharge # top left
                q_vertices[i,j,l+1,1] = max_discharge # top right
                q_vertices[i,j,l+1,2] = min_discharge # bottom right

In [ ]:
power_vertices = np.zeros((2,2,10,3))

for i in cascaded_reservoirs:
    for j in hydro_units[i]:
        for l in subregions:
            for r in vertices:
                power_vertices[i,j,l,r] = production_function(efficieny, h_vertices[i,j,l,r], q_vertices[i,j,l,r])

In [ ]:
psh.clear()
volume = psh.continuous_var_matrix(psh.reservoirs, psh.periods, name='reservoir_volume')
total_discharge = psh.continuous_var_matrix(psh.reservoirs, psh.periods, name='total_discharge_ex_ps')
hu_discharge = psh.continuous_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='discharge_hydro_units')
hu_head = psh.continuous_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='head_hydro_units')
hu_power = psh.continuous_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='power_output_hydro_units')
active_subregion = psh.binary_var_dict([(i, j, t, l) for i in psh.reservoirs for j in psh.hydro_units for t in psh.periods for l in psh.subregions], name='active_subregions')
alpha_coefficient = psh.continuous_var_dict([(i, j, t, l, r) for i in psh.reservoirs for j in psh.hydro_units for t in psh.periods for l in psh.subregions for r in psh.vertices], name='convex_combination_coefficients')
hu_start_up = psh.binary_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='start_ups_hydro_units')
hu_shut_down = psh.binary_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='shut_downs_hydro_units')
hu_state = psh.binary_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='state_hydro_units')
ps_power = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='total_power_output_pumped_storage_units')
ps_discharge = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='total_discharge_pumped_storage_units')
ps_power_gen = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='power_output_pumped_storage_units')
ps_power_pump = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='power_consumption_pumped_storage_units')
ps_discharge_gen = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='discharge_pumped_storage_units')
ps_discharge_pump = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='pumped_upstream_pumped_storage_units')
ps_state_gen = psh.binary_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='generation_state_pumped_storage_units')
ps_state_pump = psh.binary_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='pumping_state_pumped_storage_units')